# Idea
Do some training

In [7]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [8]:
import sys
repo_base_path = Path("./../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

# from py_dataset import get_all_files_df
# from py_dataset import read_in_files
from py_dataset import feature_plotting
# from py_dataset import feature_selection
# from py_dataset import net_feature_engineering
# from py_dataset import entropy_feature_engineering
# from py_dataset.classes import DataSource
from py_dataset import sys_func



In [9]:
df = pd.read_csv('all_df_top34.csv')
df.set_index("timestamp", inplace=True)

In [10]:
df["label"].unique()

array(['1_normal', nan, '2_ransomware', '3_thetick', '4_bashlite',
       '5_httpbackdoor', '6_beurk', '7_backdoor', '8_bdvl', '9_xmrig'],
      dtype=object)

### Preprocessings
- remove Null values
- drop timestamp
- labelEncode
- normalize

In [11]:
print(df.shape)
old_df = df.copy()
df.dropna(axis=0, how='any', inplace=True)

print(df.shape)

(29427, 34)
(20345, 34)


In [12]:
df.reset_index(inplace=True, drop=True)

In [13]:
label_encoder = sys_func.create_label_encoder_for_labels()
df['label'] = label_encoder.transform(df['label'])

In [14]:
df = feature_plotting.test_transform_all_to_numeric_columns(df)

In [15]:
cols_to_exclude_from_scaling = ["label"]
vectors = feature_plotting.minmax_scale_features(df,remove_outliers_iqr_all_columns=False,
                                                        cols_to_exclude_from_scaling=cols_to_exclude_from_scaling)
# print(vectors.shape)
# print(" with dropped null: ", vectors.dropna(axis=0, how='any', inplace=False).shape)

In [16]:
df["label"].unique(), vectors["label"].unique()

(array([0, 1, 2, 3, 4, 5, 6, 7, 8]), array([0, 1, 2, 3, 4, 5, 6, 7, 8]))

### Setup

In [17]:
from torch.utils.data import Dataset, DataLoader
import torch

class CustomeDataset(Dataset):
    def __init__(self, some_df:pd.DataFrame):
        
        self.x = torch.tensor(some_df.drop(columns=["label"], inplace=False).values, dtype=torch.float32)
        self.labels = torch.tensor(some_df["label"].values, dtype=torch.uint8)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # if torch.is_tensor(idx):
        #     idx = idx.tolist()
        # print(self.x[idx], self.labels[idx])
        return self.x[idx], self.labels[idx]

In [18]:
from mlp import MalwaresModelMLP
from torch.utils.data import DataLoader
import lightning
from sklearn.model_selection import train_test_split
from lightning import loggers as pl_loggers
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint

# tensorboard = pl_loggers.TensorBoardLogger('./')

# split into train and val
train, val = train_test_split(vectors, test_size=0.2)

train_dataset = CustomeDataset(train)
train_dataloader = DataLoader(train_dataset, batch_size=500, shuffle=True, num_workers=7)

val_dataset = CustomeDataset(val)
val_dataloader = DataLoader(val_dataset, batch_size=500, shuffle=False, num_workers=7)

# trainer = lightning.Trainer(fast_dev_run=100)
monitor = "Validation/Accuracy"
early_stop_callback = EarlyStopping(monitor=monitor, patience=5, verbose=True, mode="max")
checkpoint_callback = ModelCheckpoint(save_top_k=1, monitor=monitor, mode="max")

trainer = lightning.Trainer(max_epochs=150, callbacks=[early_stop_callback, checkpoint_callback], enable_checkpointing=True) #logger=None, 


model = MalwaresModelMLP(input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                      | Params
------------------------------------------------------------
0 | train_metrics | MetricCollection          | 0     
1 | val_metrics   | MetricCollection          | 0     
2 | test_metrics  | MetricCollection          | 0     
3 | cm            | MulticlassConfusionMatrix | 0     
4 | criterion     | CrossEntropyLoss          | 0     
5 | l1            | Linear                    | 1.0 K 
6 | l2            | Linear                    | 930   
7 | l3            | Linear                    | 279   
------------------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 0
ValidationEpoch/CM
 tensor([[  0,   0,   0,   0,   0,   0,   0,  89,   0],
        [  0,   0,   0,   0,   0,  90,   0,  28,   0],
        [  0,   0,   0,   0,   0,   1,   0, 188,   0],
        [  0,   0,   0,   0,   0,   2,   0,  90,   0],
        [  0,   0,   0,   0,   0,   0,   0, 111,   0],
        [  0,   0,   0,   0,   0,   0,   0, 108,   0],
        [  0,   0,   0,   0,   0,   0,   0, 105,   0],
        [  0,   0,   0,   0,   0,   1,   0, 106,   0],
        [  0,   0,   0,   0,   0,   4,   0,  77,   0]])


/home/<User>/repos/csg_is/.venv/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (33) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 24
ValidationEpoch/CM
 tensor([[30074,   212, 11233,  4454,  1007,   299,   650,   527,    17],
        [    0, 52067,     0,     0,     0,   146,     0,   105,     0],
        [ 3294,   384, 64143,  5284,  3953,  1097,  4955,   935,     0],
        [ 3966,   201, 20818, 18041,  3848,   372,  2576,   502,     0],
        [  754,   266, 11677,  3973, 28184,  2645,  2126,   886,     0],
        [    1,   293,  2636,   140,  1622, 43386,   695,  1855,     0],
        [ 1601,   222, 25901,  5527,  4060,  2590,  9389,   927,     0],
        [  208,   249,  1737,    92,   152,  1559,   119, 47111,     0],
        [    3,     0,     0,     0,     1,     6,     0,   218, 51309]])
Epoch end: Train, epoch number: 23
TrainEpoch/CM
 tensor([[30074,   212, 11233,  4454,  1007,   299,   650,   527,    17],
        [    0, 52067,     0,     0,     0,   146,     0,   105,     0],
        [ 3294,   384, 64143,  5284,  3953,  1097,  4955,   935,     0],
        [ 396

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch end: Validation, epoch number: 25
ValidationEpoch/CM
 tensor([[31869,   212, 11303,  4569,  1029,   299,   663,   527,    18],
        [    0, 54242,     0,     0,     0,   146,     0,   105,     0],
        [ 3370,   384, 66891,  5505,  4064,  1099,  5291,   935,     0],
        [ 4074,   201, 21398, 19150,  4012,   373,  2707,   502,     0],
        [  780,   266, 11896,  4216, 29668,  2659,  2237,   889,     0],
        [    1,   293,  2637,   145,  1638, 45409,   749,  1861,     0],
        [ 1625,   222, 26835,  5821,  4206,  2653, 10013,   930,     0],
        [  213,   249,  1737,    92,   154,  1567,   125, 49220,     0],
        [    3,     0,     0,     0,     1,     6,     0,   218, 53453]])
Epoch end: Train, epoch number: 24
TrainEpoch/CM
 tensor([[31869,   212, 11303,  4569,  1029,   299,   663,   527,    18],
        [    0, 54242,     0,     0,     0,   146,     0,   105,     0],
        [ 3370,   384, 66891,  5505,  4064,  1099,  5291,   935,     0],
        [ 407

Monitored metric Validation/Accuracy did not improve in the last 5 records. Best score: 0.804. Signaling Trainer to stop.


In [20]:
model.eval()

MalwaresModelMLP(
  (train_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Train/
  )
  (val_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Validation/
  )
  (test_metrics): MetricCollection(
    (MulticlassAccuracy): MulticlassAccuracy()
    (MulticlassPrecision): MulticlassPrecision()
    (MulticlassRecall): MulticlassRecall()
    (MulticlassF1Score): MulticlassF1Score(),
    prefix=Test/
  )
  (cm): MulticlassConfusionMatrix()
  (criterion): CrossEntropyLoss()
  (l1): Linear(in_features=33, out_features=30, bias=True)
  (l2): Linear(in_features=30, out_features=30, bias=True)
  (l3): Linear(in_features=30, out_features=9, bias=True)
)

In [21]:
print(checkpoint_callback.best_model_path)   # prints path to the best model's checkpoint
print(checkpoint_callback.best_model_score) # and prints it score
best_model = MalwaresModelMLP.load_from_checkpoint(checkpoint_callback.best_model_path, input_size = vectors.shape[1]-1, output_size = len(label_encoder.classes_))

trainer.test(best_model, dataloaders=val_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/home/<User>/repos/csg_is/training/lightning_logs/version_8/checkpoints/epoch=19-step=660.ckpt
tensor(0.8042, device='cuda:0')


Testing: |          | 0/? [00:00<?, ?it/s]

Epoch end: Test, epoch number: 0
TestEpoch/CM
 tensor([[340,   0,  13,  45,   6,   0,   2,   0,   0],
        [  0, 448,   0,   0,   0,   0,   0,   0,   0],
        [  7,   0, 572,  57,  21,   1,  76,   0,   0],
        [  4,   0, 101, 225,  28,   0,  19,   0,   0],
        [  1,   0,  35,  61, 289,   9,  27,   1,   0],
        [  0,   0,   0,   2,   2, 427,   5,   0,   0],
        [  3,   0, 145,  58,  18,  19, 151,   2,   0],
        [  1,   0,   0,   0,   0,   3,   1, 419,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0, 425]])
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Test/Accuracy         0.8042055368423462
      Test/F1Score          0.8072361946105957
        Test/Loss           0.5107317566871643
     Test/Precision         0.

[{'Test/Loss': 0.5107317566871643,
  'Test/Accuracy': 0.8042055368423462,
  'Test/Precision': 0.8186952471733093,
  'Test/Recall': 0.8042055368423462,
  'Test/F1Score': 0.8072361946105957,
  'TestEpoch/Accuracy': 0.8050711750984192,
  'TestEpoch/Precision': 0.8191437125205994,
  'TestEpoch/Recall': 0.8050711750984192,
  'TestEpoch/F1Score': 0.8086774349212646}]

In [22]:
x,t = next(iter(train_dataloader))

z = model(x)

torch.argmax(z, dim=0)

tensor([435,  10,  55, 323, 424, 132, 296, 420,  30])